In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

dft = pd.read_csv('PA1_train.csv')
df_train = dft.drop('id', 1)
dfd = pd.read_csv('PA1_dev.csv')
df_dev = dfd.drop('id', 1)
df_train['year'] = pd.DatetimeIndex(df_train['date']).year
df_train['month'] = pd.DatetimeIndex(df_train['date']).month
df_train['day'] = pd.DatetimeIndex(df_train['date']).day
df_dev['year'] = pd.DatetimeIndex(df_dev['date']).year
df_dev['month'] = pd.DatetimeIndex(df_dev['date']).month
df_dev['day'] = pd.DatetimeIndex(df_dev['date']).day

df_train.head()

,dummy,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price,year,month,day
0,1,7/9/2014,4,2.50,2720,10006,2.0,0,0,3,...,0,98074,47.6295,-122.042,2720,10759,5.9495,2014,7,9
1,1,7/18/2014,2,2.50,2600,5000,1.0,0,0,5,...,0,98126,47.5806,-122.379,2260,5000,6.6500,2014,7,18
2,1,7/7/2014,5,1.75,1650,3000,1.5,0,0,3,...,0,98144,47.5955,-122.306,1740,4000,4.4300,2014,7,7
3,1,4/28/2015,4,1.75,1720,9600,1.0,0,0,4,...,0,98059,47.4764,-122.155,1660,10720,3.8000,2015,4,28
4,1,9/4/2014,6,2.25,3830,11180,1.0,0,2,5,...,0,98008,47.5849,-122.113,2500,10400,8.8700,2014,9,4


In [2]:
df_train = df_train.drop('date', 1)
df_dev = df_dev.drop('date', 1)

df_columns = df_train.columns.values
list_columns = df_columns.tolist()

# original dataset 
df_cate_col = ['waterfront', 'grade', 'condition', 'zipcode']

#1. use year and month  as categorial value 

#df_cate_col = ['waterfront', 'grade', 'condition', 'zipcode', 'month', 'year']

#2. use zipcode as normal numerical data 
#df_cate_col = ['waterfront', 'grade', 'condition'] 
#print(list_columns)

for i in df_cate_col:
  list_columns.remove(i)

df_num_col = np.asarray(list_columns)
list_columns.remove('price')
df_num_col_wo_price = np.asarray(list_columns)

#statistic values
df_train_num_means = df_train[df_num_col_wo_price].mean()
df_train_num_std = df_train[df_num_col_wo_price].std()

df_train_num_max = df_train[df_num_col_wo_price].max()
df_train_num_min = df_train[df_num_col_wo_price].min()
df_train_num_range = df_train_num_max - df_train_num_min


In [3]:
df_norm_fea = [];
for i in range(len(df_num_col_wo_price)):
  norm_fea = [];
  cur_fea = df_num_col_wo_price[i];
  print(cur_fea);
  max = df_train[cur_fea].max();
  min = df_train[cur_fea].min();
  range_val = max - min;
  for j in range(len(df_train[cur_fea])):
    cur_val = df_train[cur_fea][j];
    norm = float((cur_val - min) / (range_val) )
    norm_fea.append(norm)
  df_norm_fea.append(norm_fea)

np_norm_fea = np.asarray(df_norm_fea)
dict_norm_fea = dict(zip(df_num_col_wo_price, np_norm_fea))

dummy
bedrooms


/nfs/stak/users/ohda/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in long_scalars
  # This is added back by InteractiveShellApp.init_path()


bathrooms
sqft_living
sqft_lot
floors
view
sqft_above
sqft_basement
yr_built
yr_renovated
lat
long
sqft_living15
sqft_lot15
year
month
day


In [4]:
df_dev_norm_fea = [];
for i in range(len(df_num_col_wo_price)):
  norm_fea = [];
  cur_fea = df_num_col_wo_price[i];
  max = df_dev[cur_fea].max();
  min = df_dev[cur_fea].min();
  range_val = max - min;
  for j in range(len(df_dev[cur_fea])):
    cur_val = df_dev[cur_fea][j];
    norm = float((cur_val - min) / (range_val) )
    norm_fea.append(norm)
  df_dev_norm_fea.append(norm_fea)


np_dev_norm_fea = np.asarray(df_dev_norm_fea)
dict_dev_norm_fea = dict(zip(df_num_col_wo_price, np_dev_norm_fea))

df_train_target = df_train['price']
df_dev_target = df_dev['price']


/nfs/stak/users/ohda/.local/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in long_scalars
  # Remove the CWD from sys.path while we load stuff.


In [5]:
df_train_trial = df_train[df_num_col_wo_price];
for idx in range(1, len(df_num_col_wo_price)):
  df_train_trial[df_num_col_wo_price[idx]] = dict_norm_fea[df_num_col_wo_price[idx]]


df_dev_trial = df_dev[df_num_col_wo_price]
for idx in range(1, len(df_num_col_wo_price)):
  df_dev_trial[df_num_col_wo_price[idx]] = dict_dev_norm_fea[df_num_col_wo_price[idx]]


for i in range(len(df_cate_col)):
  a = pd.get_dummies(df_train[df_cate_col[i]], prefix=df_cate_col[i])
  df_train_trial = pd.concat([df_train_trial, a], axis=1)
qq = np.zeros(df_train.shape[0])
df_train_trial.insert(20, column='grade_3', value=qq)


for i in range(len(df_cate_col)):
  a = pd.get_dummies(df_dev[df_cate_col[i]], prefix=df_cate_col[i])
  df_dev_trial = pd.concat([df_dev_trial, a], axis=1)

/nfs/stak/users/ohda/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/nfs/stak/users/ohda/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
learning_rate = [10**0, 10**-1, 10**-2, 10**-3, 10**-4, 10**-5, 10**-6, 10**-7]
# 1. changing epsilon to smaller value
epsilon = 0.01
#epsilon = 0.5

def MSE(w, X, y):
	N = X.shape[0];
	resd = np.dot(X,w) - y;
	mse = np.sum((resd**2))/(N) 
	return mse;

def get_grad(w, X, y, N, d):
  y_pred = np.dot(X,w);
  resd = y_pred - y;
  grad_vec = (2*np.dot(X.T, resd))/N
  return resd, grad_vec;

def learning(df_vals, df_target, learning_rate, max_itr):
  t = time.time(); 
  itr_cnt = 0;
  mse_norm = 100;
  d = df_vals.shape[1];
  N = df_vals.shape[0];
  w = np.zeros(d);
  y = np.array(df_target);
  X = np.array(df_vals);

  losses = [];
  mse_norms = [];

  while (mse_norm > epsilon):
    itr_cnt += 1;
    resd, grad = get_grad(w, X, y, N, d);
    w = w - (learning_rate*grad);
    loss = np.sum((resd**2))/N
    losses.append(loss);
    mse_norm = np.linalg.norm(grad);
    mse_norms.append(mse_norm)
   

  elapsed = time.time() - t;
  return (mse_norms, losses, itr_cnt, elapsed, w);


In [ ]:
print()
idx = 3
print("iteration with lower epsilon 0.01")
#print("iteration with considering value -month- also as categorical data")
#print("iteration considering -zipcode- as numerical data")
mse_norms, losses, itr_cnt, elapsed, weights = learning(df_train_trial, df_train_target, learning_rate[idx], 1000000)
cur_mse_train = MSE(weights, df_train_trial, df_train_target);
print('MSE of train data with learning rate ' + str(learning_rate[idx]) + " : ", cur_mse_train, " with iteration: ", itr_cnt);
print('iteration score: ', itr_cnt)
print('time took : ', elapsed)


iteration with lower epsilon 0.01
